In [1]:
from appgeopy import *
from my_packages import *

In [2]:
gwl_hdf5_file = "20240828_GWL_CRFP_model.h5"
# List available datasets in the HDF5 file for reference
with h5py.File(gwl_hdf5_file, "r") as hdf5_file:
    available_datasets = gwatertools.h5pytools.list_datasets(hdf5_file)
    datetime_idx = pd.to_datetime(hdf5_file["date"][...], format="%Y%m%d")

available_datasets[:5]

['ANHE/10070111/measure/daily_value',
 'ANHE/10070111/measure/model',
 'ANHE/10070121/measure/daily_value',
 'ANHE/10070121/measure/model',
 'ANHE/10070131/measure/daily_value']

In [3]:
available_stations = sorted(
    set([elem.split("/")[0] for elem in available_datasets if elem != "date"])
)
available_stations[:5]

['ANHE', 'ANNAN', 'BEIGANG', 'BOZI', 'CAICUO']

In [4]:
# select_station = available_stations[0]
for select_station in tqdm(available_stations[:]):
    # _____________________________________________
    datagroup_byStation = sorted(
        {
            "/".join(elem.split("/")[:2])
            for elem in available_datasets
            if elem.startswith(select_station)
        }
    )
    # _____________________________________________
    figsize = (11.7, 8.27)
    nrows = len(datagroup_byStation)

    fig, axes = plt.subplots(nrows=nrows, ncols=1, figsize=figsize, sharex=True)
    fig_width, fig_height = figsize
    scaling_factor = visualize.calculate_scaling_factor(fig_width, fig_height)
    # Ensure axes is an array even if there's only one subplot
    if nrows == 1:
        axes = np.atleast_1d(axes)
    else:
        axes = axes.flatten()
    # _____________________________________________
    for idx, select_datagroup in enumerate(datagroup_byStation):
        wellcode = select_datagroup.split("/")[-1]
        # _____________________________________________
        with h5py.File(gwl_hdf5_file, "r") as hdf5_file:
            # _________________________________________________________________________
            original_array = hdf5_file[
                select_datagroup + "/measure/daily_value"
            ][...]
            original_series = pd.Series(data=original_array, index=datetime_idx)
            # _________________________________________________________________________
            model_array = hdf5_file[select_datagroup + "/measure/model"][...]
            model_series = pd.Series(data=model_array, index=datetime_idx)
        # _________________________________________________________________________
        ax = axes[idx]
        visualize.base_plot(
            ax=ax,
            data=original_series,
            xlabel=" ",
            scaling_factor=scaling_factor,
            label="Orginal",
            color="black",
            fontsize_base=14,
        )
        visualize.base_plot(
            ax=ax,
            data=model_series,
            xlabel=" ",
            scaling_factor=scaling_factor,
            label="Model",
            color="orangered",
            linestyle="--",
            fontsize_base=14,
        )

        ax.legend(loc="lower left", frameon=False, labelspacing=0.05)
        ax.grid(which="major", axis="x", color="lightgrey", alpha=0.5)
        ax.text(
            x=0.9,
            y=0.95,
            s=wellcode,
            transform=ax.transAxes,
            fontweight="bold",
            color="blue",
            fontsize=10 * scaling_factor,
        )
        ax.xaxis.set_major_locator(mdates.YearLocator())
        # Optionally, format the date to show only the year
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
        # Set the minor ticks to be at June 1st of each year
        ax.xaxis.set_minor_locator(mdates.MonthLocator(bymonth=6, bymonthday=30))
        # Configure minor ticks to make them visible
        ax.tick_params(axis='x', which='major', length=8, color='gray')
        ax.tick_params(axis='x', which='minor', length=5, color='gray')

    # Rotate the x-axis labels for better readability, if needed
    plt.setp(ax.get_xticklabels(), rotation=90, ha="center")
    fig.suptitle(
        select_station,
        fontsize=12 * scaling_factor * 1.5,
        y=0.925,
        fontweight="bold",
    )
    fig.text(
        0.04,
        0.5,
        "Groundwater Levels (m)",
        va="center",
        rotation="vertical",
        fontsize=14 * scaling_factor,
        fontweight="bold",
    )
    visualize.save_figure(fig=fig, savepath=os.path.join("Figures", select_station+".png"))
    plt.close()

100%|████████████████████████████████████████████████████████████████████████████████| 104/104 [03:05<00:00,  1.78s/it]
